In [1]:
import os
os.getcwd()

'/home/whitesox/Dropbox (UNC Charlotte)/itcs4010-tweets'

In [2]:
# see https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
import json

# beer tweets
file = './data/tweets-2018-07-15-10-55-49.json'

with open(file, 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print

{
    "created_at": "Sun Jul 15 14:55:50 +0000 2018",
    "id": 1018509449156177923,
    "id_str": "1018509449156177923",
    "text": "The sports bars in Makati poblacion must be jam-packed with beer guzzling afam hotties right now for the finals liv\u2026 https://t.co/qgtsY1nM5s",
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
    "truncated": true,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 1898572387,
        "id_str": "1898572387",
        "name": "\u00a8\u032e",
        "screen_name": "jasminnism",
        "location": "Philippines",
        "url": null,
        "description": null,
        "translator_type": "none",
        "protected": false,
        "verified": false,
        "followers_count": 439,
        "friends_count": 252,
        "listed_count": 7,
    

## Exploring the data

Let's now consider getting some of the information.

In [3]:
print(tweet["id"]) # unique tweet id

1018509449156177923


In [4]:
print(tweet["user"]["screen_name"]) # screen_name (handle)

jasminnism


In [5]:
print(tweet["user"]["location"]) # profile location (open-ended string)

Philippines


In [6]:
print(tweet["text"]) # body of the tweet

The sports bars in Makati poblacion must be jam-packed with beer guzzling afam hotties right now for the finals liv… https://t.co/qgtsY1nM5s


In [7]:
print(tweet["extended_tweet"]["full_text"]) # full body of the tweet

The sports bars in Makati poblacion must be jam-packed with beer guzzling afam hotties right now for the finals live viewing. *sigh* LOL.

Anyway, goodluck Team #CRO⁠ ⁠ ! 

#WorldCup⁠ ⁠ 
#FRACRO

(☞ﾟヮﾟ)☞ 🇭🇷🇭🇷🇭🇷


## Send to Pandas Dataframe

We'll now get the body of the tweets and append it to a pandas dataframe.

In [8]:
def get_tweets(tweets_json):
    
    tweets_list = []

    with open(file, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            
            # take body of tweet
            text = tweet['text']
            
            if 'extended_tweet' in tweet:
            # use extended tweet if it exists
                text = tweet['extended_tweet']['full_text']
            
            # append into list
            tweets_list.append(text)

    return tweets_list

In [9]:
import pandas as pd

# Flatten the tweets and store in `tweets`
tweets = get_tweets(file)

# Create a DataFrame from `tweets`
wc_tweets = pd.DataFrame(tweets, columns=['text'])

# print out first five tweets
print(wc_tweets.values[0:5])

[['The sports bars in Makati poblacion must be jam-packed with beer guzzling afam hotties right now for the finals live viewing. *sigh* LOL.\n\nAnyway, goodluck Team #CRO\u2060 \u2060 ! \n\n#WorldCup\u2060 \u2060 \n#FRACRO\n\n(☞ﾟヮﾟ)☞ 🇭🇷🇭🇷🇭🇷']
 ['8am got my beer ready, #Croatia jersey is on. #WorldCupFinal is on!!!! I’m fucking ready for #Hrvatska to take this trophy home!!!!!']
 ["RT @PWXPURE: IT'S OFFICIAL:\nPWX Wrestling Pure Four\nEffy vs Slim J\n\nThis Sunday July 15\nHebron Hall\nTix: https://t.co/WdMtP2rPBD\nCraft Beer…"]
 ['RT @budargentina: Fin de una gran Rusia, fin de una gran fiesta #Budweiser Official Beer of The FIFA World Cup™ 💪 🔥 https://t.co/wMbDu6Cjhc']
 ['Gutted seeing build up to final now. Not sure what we would have done but I’d have had the garden decked out ready with 101 chairs, bbq on non-stop, wheelie bin full of beer on ice, wide screen in garden!']]


In [10]:
import numpy as np

world_cup = wc_tweets['text'].str.contains('World Cup', case = False)

print("Percent of 'World Cup' tweets:", np.sum(world_cup) * 100 / wc_tweets.shape[0])

Percent of 'World Cup' tweets: 5.813953488372093


In [11]:
keywords = ['World Cup', 'worldcup']

# https://stackoverflow.com/questions/26577516/how-to-test-if-a-string-contains-one-of-the-substrings-in-a-list
world_cup = wc_tweets['text'].str.contains('|'.join(keywords), case = False)

print("Percent of 'World Cup' tweets:", np.sum(world_cup) * 100 / wc_tweets.shape[0])

Percent of 'World Cup' tweets: 12.325581395348838


In [12]:
fire_emoji = wc_tweets['text'].str.contains('🔥', case = False)

print("Percent of '🔥' tweets:", np.sum(fire_emoji) * 100 / wc_tweets.shape[0])

Percent of '🔥' tweets: 0.46511627906976744


## Pre-processing and text

http://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#.W8dUIBNKjUI

In [13]:
import re

def pre_process(text):  
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text = re.sub('[^a-zA-Z]+', ' ', text)
    
    return text

wc_tweets['clean_text'] = wc_tweets['text'].apply(pre_process)

In [14]:
wc_tweets['text'][0]

'The sports bars in Makati poblacion must be jam-packed with beer guzzling afam hotties right now for the finals live viewing. *sigh* LOL.\n\nAnyway, goodluck Team #CRO\u2060 \u2060 ! \n\n#WorldCup\u2060 \u2060 \n#FRACRO\n\n(☞ﾟヮﾟ)☞ 🇭🇷🇭🇷🇭🇷'

In [15]:
wc_tweets['clean_text'][0]

'the sports bars in makati poblacion must be jam packed with beer guzzling afam hotties right now for the finals live viewing sigh lol anyway goodluck team cro worldcup fracro '

## Word counts

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

#get the text column 
docs=wc_tweets['clean_text'].tolist()
 
#create a vocabulary of words, 
cv=CountVectorizer()
tf_counts=cv.fit_transform(docs)

# size
print('sparse matrix shape: ',tf_counts.shape)

# non-zero count
print('sparse matrix shape: ',tf_counts.nnz)

# sparsity
print('sparsity %: ',(100.0 * tf_counts.nnz / (tf_counts.shape[0] * tf_counts.shape[1])))

sparse matrix shape:  (860, 4380)
sparse matrix shape:  14589
sparsity %:  0.3873048741637464


In [17]:
# function to get counts as dataframe
def get_df_counts(tf_counts, cv):
    occ = np.asarray(tf_counts.sum(axis=0)).ravel().tolist()
    counts_df = pd.DataFrame({'term': cv.get_feature_names(), 'occurrences': occ})    
    return counts_df

In [18]:
get_df_counts(tf_counts, cv).sort_values(by='occurrences', ascending=False).head(20)

occurrences     term
320           627     beer
740           534       co
1807          534    https
3737          506      the
123           263      and
3811          240       to
3223          219       rt
2692          203       on
2663          173       of
1366          167      for
1873          156       in
2042          156     just
4336          133      you
239           128    badge
1935          127       it
2528          127       my
3964          126  untappd
1096          126   earned
195           116       at
557           111       by

In [19]:
cv2=CountVectorizer(stop_words='english')
tf_counts2=cv2.fit_transform(docs)

# size
print('sparse matrix shape: ',tf_counts2.shape)

# non-zero count
print('sparse matrix shape: ',tf_counts2.nnz)

# sparsity
print('sparsity %: ',(100.0 * tf_counts2.nnz / (tf_counts2.shape[0] * tf_counts2.shape[1])))

sparse matrix shape:  (860, 4178)
sparse matrix shape:  9815
sparsity %:  0.27316397074376303


In [ ]:
get_df_counts(tf_counts2, cv2).sort_values(by='occurrences', ascending=False).head(20)

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the tf-idf feature matrix
tfidf = TfidfVectorizer(stop_words='english', use_idf=True)
feature_matrix = tfidf.fit_transform(wc_tweets['clean_text'])

# Show tf-idf feature matrix
feature_matrix.toarray()

In [ ]:
# Show tf-idf feature matrix
tfidf.get_feature_names()

In [ ]:
# Create data frame
tfidf_df = pd.DataFrame(feature_matrix.toarray(), columns=tfidf.get_feature_names())

In [ ]:
tfidf_df.head()

http://www.ultravioletanalytics.com/blog/tf-idf-basics-with-pandas-scikit-learn

In [ ]:
weights = np.asarray(feature_matrix.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tfidf.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)